In [3]:
import sys, os
sys.path.append(os.path.realpath("/home/saqibm128/miniconda3/envs/keras/lib/python3.7/site-packages"))
sys.path.append(os.path.realpath("../"))

import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import data_reader as read
import pandas as pd
from seq2seq.models import Seq2Seq
reload(read)

edfRawData = read.EdfDataset("dev_test", "01_tcp_ar", num_files=20)
data = read.EdfFFTDatasetTransformer(edfRawData, window_size=pd.Timedelta(seconds=1), precache=True)

model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=10, output_length=8, output_dim=20, depth=4)
model.compile(loss='mse', optimizer='rmsprop')

starting precache job with: 8 processes
retrieving: 0


In [ ]:
data[0]